# Psycopg

Psycopg is a postgres adapter for python programming language.

In [59]:
import time
import docker
import psycopg2

client = docker.from_env()
container_name = "psycopg_examples"
container_port = 5432

The following cell runs postgres in a docker container or reruns it if necessary.

In [60]:
def check_container(name: str) -> bool:
    containters = [
        container.name 
        for container in client.containers.list()
    ]
    return name in containters


try:
    container = client.containers.get(container_name)
    container.stop()
    while check_container(container_name):
        time.sleep(1)

except docker.errors.NotFound:
    pass
finally:
    container = client.containers.run(
        "postgres:17.4",
        name=container_name,
        environment={
            "POSTGRES_PASSWORD": "password",
        },
        ports={f"{container_port}/tcp": 5432},
        detach=True,
        remove=True
    )

## Execute

To run anything in postgres you need to:

- Create a connection: `psycopg2.connect()`.
- Create cursor: `connection.cursor()`.
- Execute your query: `cursor.execute()`.
- Get data from cursor: `cursor.fetchall`, `cursor.fetachone`, `cursor.description` etc.

---

The following cell connects to the database.

In [73]:
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="password",
    host="localhost",
    port=container_port
)

The next code executes the query in the database and loads the results and the name of the column.

In [79]:
with connection.cursor() as cursor:
    cursor.execute("SELECT 50 AS value1, 'test' AS value2;")
    columns = cursor.description
    data = cursor.fetchall()

print("Columns", columns)
print("Data", data)

Columns (Column(name='value1', type_code=23), Column(name='value2', type_code=25))
Data [(50, 'test')]
